# Obtención nombre y equipo actual jóvenes promesas.

❗ Se procede a buscar jugadores con un valor actual inferior a **1 millón de euros** y que tengan una previsible progresión.

➡ Para ello, se realizará un Web Scraping de la página Sofifa, la cual nos permite establecer un precio máximo del jugador y, por otro lado, el posible proreso del mismo.

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests

pd.options.display.max_columns = None

1️⃣ Establecemos el scrapeo.

In [2]:
url = 'https://sofifa.com/players?type=all&oah=70&ptl=80&vlh=1000000'
html = requests.get(url)
soup = BeautifulSoup(html.content, "html.parser")
html.status_code

200

2️⃣ Dentro de la página, marcamos la información que deseamos obtener.

In [3]:
productos = soup.find_all("td", {"class": "col-name"})

3️⃣ Creamos un diccionario al que añadir los datos escrapeados para, posteriormente, convertirlo en un dataframe con el que trabajar.

In [4]:
jugadores = {'jugador': []}

In [5]:
for i in productos:
    jugadores['jugador'].append(i.find('a').get('href'))                           

In [6]:
jugadores_top = pd.DataFrame(jugadores)
jugadores_top.head()

,jugador
0,/player/267750/renzo-sanchez/220057/
1,/team/111325/club-nacional-de-football/
2,/player/266437/dimitrios-keramitsis/220057/
3,/team/52/roma/
4,/player/266031/ruben-iranzo-lendinez/220057/


❗ Como se observa en el dataframe anterior, el nombre del juagdor actual y su equipo aparecen acompañados de letras y números que no son de interés. Por este motivo, realizaremos una limpieza empleando regex.

4️⃣ Aplicamos regex al nombre del jugador, limpiándolo y obteniéndolo de manera individual. Además, como en las celdas del nombre del equipo actual no se cumple el regex aparecerá NaN, se procederá a eliminarlas para limpiarlas posteriormente.

In [10]:
jugadores_top['jug'] = jugadores_top['jugador'].str.extract(r'(?<=\/player\/\d\d\d\d\d\d\/)(.*)(?=\/220057)')

In [11]:
jugadores_top.head()

,jugador,jug
0,/player/267750/renzo-sanchez/220057/,renzo-sanchez
1,/team/111325/club-nacional-de-football/,NaN
2,/player/266437/dimitrios-keramitsis/220057/,dimitrios-keramitsis
3,/team/52/roma/,NaN
4,/player/266031/ruben-iranzo-lendinez/220057/,ruben-iranzo-lendinez


In [12]:
jugadores_fichar = jugadores_top.dropna()
jugadores_fichar = jugadores_fichar.reset_index(drop=True)
jugadores_fichar.head()

,jugador,jug
0,/player/267750/renzo-sanchez/220057/,renzo-sanchez
1,/player/266437/dimitrios-keramitsis/220057/,dimitrios-keramitsis
2,/player/266031/ruben-iranzo-lendinez/220057/,ruben-iranzo-lendinez
3,/player/264179/pelayo-gonzalez-rey/220057/,pelayo-gonzalez-rey
4,/player/263640/antoni-djibu-milambo/220057/,antoni-djibu-milambo


5️⃣ Realizamos el mismo procedimiento seguido para el nombre del jugador con los equiposactuales.

In [13]:
jugadores_top['team1'] = jugadores_top['jugador'].str.extract(r'(?<=\/team\/)(.*)(?=\/)')

In [14]:
jugadores_top.head()

,jugador,jug,team1
0,/player/267750/renzo-sanchez/220057/,renzo-sanchez,NaN
1,/team/111325/club-nacional-de-football/,NaN,111325/club-nacional-de-football
2,/player/266437/dimitrios-keramitsis/220057/,dimitrios-keramitsis,NaN
3,/team/52/roma/,NaN,52/roma
4,/player/266031/ruben-iranzo-lendinez/220057/,ruben-iranzo-lendinez,NaN


In [15]:
jugadores_top['team'] = jugadores_top['team1'].str.extract(r'(?<=\d\/)(.*)')

In [16]:
jugadores_top.head()

,jugador,jug,team1,team
0,/player/267750/renzo-sanchez/220057/,renzo-sanchez,NaN,NaN
1,/team/111325/club-nacional-de-football/,NaN,111325/club-nacional-de-football,club-nacional-de-football
2,/player/266437/dimitrios-keramitsis/220057/,dimitrios-keramitsis,NaN,NaN
3,/team/52/roma/,NaN,52/roma,roma
4,/player/266031/ruben-iranzo-lendinez/220057/,ruben-iranzo-lendinez,NaN,NaN


In [17]:
equipos = jugadores_top.drop(['jug', 'jugador','team1'], axis=1)
equipos.head()

,team
0,NaN
1,club-nacional-de-football
2,NaN
3,roma
4,NaN


In [18]:
equip = equipos.dropna()
equip = equip.reset_index(drop=True)
equip.head()

,team
0,club-nacional-de-football
1,roma
2,valencia-cf
3,real-oviedo
4,feyenoord


6️⃣ En definitiva, obtendremos un dataframe con la información escrapeada (jugador), otro con el nombre del jugador limpio (jug) y, por último, el del equipo actual (team). En este caso, el de la informació escrapeada es meramente ilustrativo y se procederá a su eliminación.

In [19]:
jugador_fichar = pd.concat([jugadores_fichar, equip], axis=1)
jugador_fichar.head()

,jugador,jug,team
0,/player/267750/renzo-sanchez/220057/,renzo-sanchez,club-nacional-de-football
1,/player/266437/dimitrios-keramitsis/220057/,dimitrios-keramitsis,roma
2,/player/266031/ruben-iranzo-lendinez/220057/,ruben-iranzo-lendinez,valencia-cf
3,/player/264179/pelayo-gonzalez-rey/220057/,pelayo-gonzalez-rey,real-oviedo
4,/player/263640/antoni-djibu-milambo/220057/,antoni-djibu-milambo,feyenoord


In [20]:
jugador_fichar = jugador_fichar.drop(['jugador'], axis=1)
jugador_fichar.head()

,jug,team
0,renzo-sanchez,club-nacional-de-football
1,dimitrios-keramitsis,roma
2,ruben-iranzo-lendinez,valencia-cf
3,pelayo-gonzalez-rey,real-oviedo
4,antoni-djibu-milambo,feyenoord


7️⃣ Renombramos las columnas del dataframe final para darle formato y lo guardamos.

In [21]:
nuevas_columnas = {
    'jug' : 'Nombre jugador', 
    'team': "Equipo actual"}

In [22]:
jugador_fichar.rename(columns = nuevas_columnas, inplace = True)
jugador_fichar.head(2)

,Nombre jugador,Equipo actual
0,renzo-sanchez,club-nacional-de-football
1,dimitrios-keramitsis,roma


In [23]:
jugador_fichar.to_csv('../../data/promesas/nombres_equipos.csv')